#### 포인트 클라우드에서 Frustum에 해당하는 포인트만 남기는 법
이하 코드에서는 KITTI PointCloud 처리 방법을 살펴본다.

In [52]:
import argparse
import pdb
import cv2
import numpy as np
import os
from tqdm import tqdm
import sys
# CUR = os.path.dirname(os.path.abspath(__file__))
# sys.path.append(CUR)
CUR = 'C:\DDrive\PointPillars'
sys.path.append('.')
from utils import read_points, write_points, read_calib, read_label, \
    write_pickle, get_points_num_in_bbox, points_in_bboxes_v2
    #remove_outside_points

In [53]:
# Copied from https://github.com/open-mmlab/mmdetection3d/blob/f45977008a52baaf97640a0e9b2bbe5ea1c4be34/mmdet3d/core/bbox/box_np_ops.py#L609
def projection_matrix_to_CRT_kitti(proj):
    """
        (1) proj 행렬에서 CR, CT를 분리한 다음에
        (2) CR에서 C가 상삼각행렬 인 특성을 이용해, QR 분해를 시도한다.
            - 여기서 왜 RinvCinv 즉 CR 역행렬을 구하는지??? 
            - R도 당연 분리가 가능하다.
        (3) C가 분리되면 Cinv @ CT로 부터 T도 구한다.
        (4) C, R, T 리턴
    """
    """Split projection matrix of kitti.
    P = C @ [R|T]
    C is upper triangular matrix, so we need to inverse CR and use QR
    stable for all kitti camera projection matrix.
    Args:
        proj (p.array, shape=[4, 4]): Intrinsics of camera.
    Returns:
        tuple[np.ndarray]: Splited matrix of C, R and T.
    """

    CR = proj[0:3, 0:3]
    CT = proj[0:3, 3]
    RinvCinv = np.linalg.inv(CR)
    Rinv, Cinv = np.linalg.qr(RinvCinv)
    C = np.linalg.inv(Cinv)
    R = np.linalg.inv(Rinv)
    T = Cinv @ CT
    return C, R, T

In [58]:
# Copied from https://github.com/open-mmlab/mmdetection3d/blob/f45977008a52baaf97640a0e9b2bbe5ea1c4be34/mmdet3d/core/bbox/box_np_ops.py#L661
def get_frustum(bbox_image, C, near_clip=0.001, far_clip=100):
    """Get frustum corners in camera coordinates.
    Args:
        bbox_image (list[int]): box in image coordinates.
        C (np.ndarray): Intrinsics.
        near_clip (float, optional): Nearest distance of frustum.
            Defaults to 0.001.
        far_clip (float, optional): Farthest distance of frustum.
            Defaults to 100.
    Returns:
        np.ndarray, shape=[8, 3]: coordinates of frustum corners.
    """
    fku = C[0, 0]
    fkv = -C[1, 1]
    u0v0 = C[0:2, 2]
    z_points = np.array(
        [near_clip] * 4 + [far_clip] * 4, dtype=C.dtype)[:, np.newaxis]
    """
    >>> z_points
    [[1.e-03]
    [1.e-03]
    [1.e-03]
    [1.e-03]
    [1.e+02]
    [1.e+02]
    [1.e+02]
    [1.e+02]]
    """
    b = bbox_image
    """
    [0, 0, 1224, 370]
    """
    box_corners = np.array(
        [[b[0], b[1]], [b[0], b[3]], [b[2], b[3]], [b[2], b[1]]], dtype=C.dtype)


In [59]:
# Modified from https://github.com/open-mmlab/mmdetection3d/blob/f45977008a52baaf97640a0e9b2bbe5ea1c4be34/mmdet3d/core/bbox/box_np_ops.py#L609
def remove_outside_points(points, r0_rect, tr_velo_to_cam, P2, image_shape):
    """Remove points which are outside of image.
    Args:
        points (np.ndarray, shape=[N, 3+dims]): Total points.
        rect (np.ndarray, shape=[4, 4]): Matrix to project points in
            specific camera coordinate (e.g. CAM2) to CAM0.
        Trv2c (np.ndarray, shape=[4, 4]): Matrix to project points in
            camera coordinate to lidar coordinate.
        P2 (p.array, shape=[4, 4]): Intrinsics of Camera2.
        image_shape (list[int]): Shape of image.
    Returns:
        np.ndarray, shape=[N, 3+dims]: Filtered points.
    """
    # 5x faster than remove_outside_points_v1(2ms vs 10ms)
    C, R, T = projection_matrix_to_CRT_kitti(P2)
    image_bbox = [0, 0, image_shape[1], image_shape[0]]
    frustum = get_frustum(image_bbox, C)
    frustum -= T
    frustum = np.linalg.inv(R) @ frustum.T
    frustum = points_camera2lidar(frustum.T[None, ...], tr_velo_to_cam, r0_rect) # (1, 8, 3)
    group_rectangle_vertexs_v = group_rectangle_vertexs(frustum)
    frustum_surfaces = group_plane_equation(group_rectangle_vertexs_v)
    indices = points_in_bboxes(points[:, :3], frustum_surfaces) # (N, 1)
    points = points[indices.reshape([-1])]
    return points

In [60]:
def create_data_info_pkl(data_root, data_type, prefix, label=True, db=False):
    sep = os.path.sep # OS별로 파일 경로는 구분하는 기준, 윈도우에서는 '/'
    print(f"Processing {data_type} data..")
    ids_file = os.path.join(CUR, 'dataset', 'ImageSets', f'{data_type}.txt')

    with open(ids_file, 'r') as f:
        ids = [id.strip() for id in f.readlines()]
    
    split = 'training' if label else 'testing'

    kitti_infos_dict = {}

    if db: # 학습할 때만 생성한다?
        kitti_dbinfos_train = {}
        db_points_saved_path = os.path.join(data_root, f'{prefix}_gt_database')
        os.makedirs(db_points_saved_path, exist_ok=True)

    for id in tqdm(ids): # train 3712, val 3769, test 
        cur_info_dict={}
        img_path = os.path.join(data_root, split, 'image_2', f'{id}.png')
        lidar_path = os.path.join(data_root, split, 'velodyne', f'{id}.bin')
        calib_path = os.path.join(data_root, split, 'calib', f'{id}.txt') 
        #print(lidar_path.split(sep)[-3:]) # / 단위로 분리한 다음에 끝에 3개 path를 분할한다.
        # e.g. ['training', 'velodyne', '000000.bin']
        cur_info_dict['velodyne_path'] = sep.join(lidar_path.split(sep)[-3:])

        img = cv2.imread(img_path)    
        image_shape = img.shape[:2]

        cur_info_dict['image'] = {
            'image_shape': image_shape,
            'image_path': sep.join(img_path.split(sep)[-3:]), 
            'image_idx': int(id),
        }

        calib_dict = read_calib(calib_path)
        cur_info_dict['calib'] = calib_dict    

        lidar_points = read_points(lidar_path)
        
        # 본 코드에서 부분. 이미지 밖의 포인트를 제거
        reduced_lidar_points = remove_outside_points(
            points=lidar_points, 
            r0_rect=calib_dict['R0_rect'], 
            tr_velo_to_cam=calib_dict['Tr_velo_to_cam'], 
            P2=calib_dict['P2'], 
            image_shape=image_shape)   

In [61]:
def main(args):
    data_root = args.data_root
    prefix = args.prefix

    ## 1. train: create data infomation pkl file && create reduced point clouds 
    ##           && create database(points in gt bbox) for data aumentation
    # pkl 파일 생성, 감소된 포인트 클라우드 생성, 데이터 증강을 위한 db(GT 박스 안의 포인트) 생성
    kitti_train_infos_dict = create_data_info_pkl(data_root, 'train', prefix, db=True)

    ## 2. val: create data infomation pkl file && create reduced point clouds
    kitti_val_infos_dict = create_data_info_pkl(data_root, 'val', prefix)
    
    ## 3. trainval: create data infomation pkl file
    kitti_trainval_infos_dict = {**kitti_train_infos_dict, **kitti_val_infos_dict}
    saved_path = os.path.join(data_root, f'{prefix}_infos_trainval.pkl')
    write_pickle(kitti_trainval_infos_dict, saved_path)

    ## 4. test: create data infomation pkl file && create reduced point clouds
    kitti_test_infos_dict = create_data_info_pkl(data_root, 'test', prefix, label=False)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Dataset infomation')
    parser.add_argument('--data_root', default='C:/DDrive/000_Dataset/kitti', 
                        help='your data root for kitti')
    parser.add_argument('--prefix', default='kitti', 
                        help='the prefix name for the saved .pkl file')
    args = parser.parse_args(args=[]) # 주피터 노트북에서 argparse 사용하려면, () -> args=[]

    main(args)

Processing train data..


  0%|          | 0/3712 [00:00<?, ?it/s]

[0, 0, 1224, 370]


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'